In [1]:
#Chapter 2 Pattern recognition 
#imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import binom,gamma
from scipy.stats import multivariate_normal

In [2]:
m = np.array([0,0])
cov = np.array([[1,3.0/5],[3.0/5,2]])

In [3]:
#Bernoulli distribution
#Binary variables
def bern(x,prob):
    a = np.power(prob,x)
    b = np.power(1-prob,1-x)
    return a*b

def bern_ML_mean(D):
    count = sum(D)
    return float(count)/len(D)

#Binomial Distribution
#Number of m observations of x = 1 in a data set of size N
def bino(m,N,prob):
    coef = binom(N,m)
    a = np.power(prob,m)
    b = np.power(1-prob,N-m)
    return coef*a*b

def bino_m_var(N,prob):
    mean = N*prob
    var = N*prob(1-prob)
    return mean,var

#Beta distribution
#conjugate prior for bino dist
def beta(mu,a,b):
    coef = float(gamma(a+b))/(gamma(a)*gamma(b))
    aa = np.power(mu,a-1)
    bb = np.power(1-mu,b-1)
    return coef*aa*bb

def beta_mean_var(a,b):
    mean = float(a)/(a+b)
    var = float(a*b)/((np.power(a+b,2)*(a+b+1)))
    return mean,var

#Gaussian Dist-Most
def gauss(x,mu,sig2):
    coef = 1.0/(np.power(2*np.pi*sig2,.5))
    diff = x-mu
    exp = np.exp((-1.0/(2*sig2))*(np.power(diff,2)))
    return coef*exp

def plot_gauss(m,std2):
    #std = np.power(cov,.5)
    x = np.arange(m-5,m+5,.1)
    y = gauss(x,m,std2)
    plt.plot(x,y)
    plt.show()
    

#mahalanobis dist
def maha_dist(x,mu,cov):
    diff = x-mu
    ret = np.matmul(diff,np.linalg.inv(cov))
    ret = np.matmul(ret,diff.T)
    return ret.diagonal()
    
#multi gauss
def multi_gauss(x,mu,cov):
    D = len(mu)
    a = 1.0/(np.power(2*np.pi,D/2.0))

    b = 1.0/(np.power(np.linalg.det(cov),.5))

    c = np.exp((-.5)*(maha_dist(x,mu,cov)))
    return a*b*c

#plots heat map of multi_gauss
def plot_multi_gauss(x,y,mu,covar):
    nbins = 50
    xx,yy = np.mgrid[x.min():x.max():nbins*1j,y.min():y.max():nbins*1j]
    xxyy = np.vstack([xx.flatten(), yy.flatten()]).T
    zz = multi_gauss(xxyy,mu,covar)
    plt.pcolormesh(xx,yy,zz.reshape(xx.shape))
    plt.show()
    
#returns eignvalues and vectors of matrix
def get_local_multi_gauss(mu,covar):
    return np.linalg.eig(covar)

#return conditional gauss parameters for 2d gauss
#p(xa|xb)
def conditional_gauss(mu,covar,xab,x_is_ab):
    ma = mu[0]
    mb = mu[1]
    covaa = covar[0][0]
    covab = covar[0][1]
    covba = covar[1][0]
    covbb = covar[1][1]

    if (x_is_ab == "b"):
        rtnmu = ma+(((covab)*(np.power(covbb,-1)))*(xab-mb))
        rtncov = covaa-((covab)*(np.power(covbb,-1))*(covba))
    else:
        rtnmu = mb+(((covba)*(np.power(covaa,-1)))*(xab-ma))
        rtncov = covbb-((covba)*(np.power(covaa,-1))*(covab))
    
    return rtnmu,rtncov

#returns marginal gauss parameters for 2d gauss
def marginal_gauss(mu,covar,x_is_ab):
    ma = mu[0]
    mb = mu[1]
    covaa = covar[0][0]
    covbb = covar[1][1]
    if (x_is_ab == "a"):
        return ma,covaa
    else:
        return mb,covbb